In [7]:
# загрузка необходимых библиотек
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install datetime

In [8]:
# Импорт необходимых библиотек
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime

In [10]:
def get(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return resp

def scrape_website(html):
    soup = BeautifulSoup(html, 'html.parser')

    links = [link.text for link in soup.find_all('a')]
    images = [image.get('src') for image in soup.find_all('img')]
    scripts = [script.text for script in soup.find_all('script')]
    styles = [style.get('href') for style in soup.find_all('link')]

    # Используем множество для хранения уникальных IP-адресов
    ips_set = set(re.findall(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', html))
    ips = list(ips_set)

    max_length = max(len(links), len(images), len(scripts), len(styles), len(ips))

    # Заполняем массивы до максимальной длины значениями None
    links += [None] * (max_length - len(links))
    images += [None] * (max_length - len(images))
    scripts += [None] * (max_length - len(scripts))
    styles += [None] * (max_length - len(styles))
    ips += [None] * (max_length - len(ips))

    # Создаем датасет
    data = {'link': links, 'image': images, 'script': scripts, 'style': styles, 'ip': ips}
    df = pd.DataFrame(data)

    return df

while True:
    url = input("Введите URL сайта для парсинга (или введите 'exit' для завершения): ")

    if url.lower() == 'exit':
        print("Программа завершена.")
        break

    response = get(url)
    html = response.text

    df = scrape_website(html)

    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"website_data_{current_time}.csv"

    df.to_csv(file_name, index=True)

    print(f"Данные о ссылках, изображениях, скриптах, стилях и уникальных IP-адресах сохранены в файл {file_name}\n")

Введите URL сайта для парсинга (или введите 'exit' для завершения): exit
Программа завершена.
